In [1]:
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models.BayesianModel import BayesianModel


# Método de Eliminación de Variables aplicado a Redes Bayesianas para predecir la Probabilidad de un Ataque Cardíaco

## 1. Método de eliminación de variables

Es un método de inferencia exacta aplicado en modelos gráficos probabilísticos (PGM), un ejemplo de PGM es el modelo de redes bayesianas. Realizar operaciones típicas como el cálculo de una probabilidad condicional sobre un grafo acíclico dirigido (DAG) puede resultar costoso computacionalmente. Existen dos enfoques principales, uno de ellos enfrenta el problema mediante heurísticas para encontrar soluciones aproximadas al problema, de esta forma, los tiempos de cálculo se ven reducidos considerablemente, por otro lado el segundo enfoque busca realizar una inferencia exacta, es decir, no aproxima la solución, sino que encuentra la óptima, esto último será más o menos costoso dependiendo del algoritmo utilizado.  
El método de eliminación de variables es un algoritmo de memorización, es decir, evita repetir cálculos guardando los resultados calculados en memoria, o bien, va calculando un resultado total de forma incremental. La estructura de datos que va almacenando la información recopilada de los cálculos previos se denomina **Factor**, sobre este se realizan dos operaciones elementales, una es la suma sobre una de sus variables y la otra es el producto de factores, el cuál funciona de forma muy similar a la operación JOIN en bases de datos (se cruzan las tablas dada una variable en común). Para cada variable inicial en la operación a realizar se tiene un factor, cada factor corresponde a la probabilidad de la variable dadas las variables ocultas, cuando se realizan producto de factores obtenemos como resultado un factor más completo el cuál puede representar la probabilidad de un conjunto de variables dadas otras variables ocultas.   
La complejidad del método de eliminación de variables depende del ordenamiento de las sumatorias sobre las variables ocultas, el problema de ordenamiento óptimo es un problema NP-HARD, por lo que se suelen utilizar 3 criterios básicos para realizar un ordenamiento adecuado que permita minimizar la cantidad de cálculos a realizar:  
  
1. Min-neighbors: Escoger la variable con el menor número de variables dependientes.
2. Min-weight: Escoger variables para minimar el producto de las cardinalidades de las variables dependientes.
3. Min-fill: Escoger vértices para minimizar el tamaño del factor que será añadido al grafo.

Dado un orden, se procede a remplazar cada una de las probabilidades presentes en un cálculo de probabilidad por su factor correspondiente y se operan los factores más profundos en las sumatorias, cada sumatoria reducirá en 1 la cantidad de variables dependientes del factor acumulado hasta ese momento, por lo que "se elimina una variable", de aquí proviene el nombre que se le da al algoritmo.  

Ref: https://ermongroup.github.io/cs228-notes/inference/ve/
  
En el próximo apartado se resolverán consultas sobre una red que modela la probabilidad de tener un ataque cardíaco antes de los 70 dado diferentes factores.  

## 2. Consultas

#### Notación
$f_C(C)$ = Factor de la variable C que depende de C (variable no conocida).  
$f_C$ = Factor de la variable C conocido, por lo tanto, es una constante.  
$f_{C}(C,E)$ = Factor de la variable C que depende de C y E (variable no conocida).  
$f_{CE}(C,E)$ = Factor resultante del cruce del factor de la variable C y la variable E (Similar a Join en una base de datos).  
$f_{C\overline{E}}(C)$ = suma (marginalización) del Factor CE sobre E (suma sobre todos los valores de E para valores iguales de C), por lo que solo depende la variable C.  

$A$ = Ataque cardíaco  
$F$ = Fumador  
$C$ = Colesterol alto  
$P$ = Presión alta  

### Consulta a 
$$P(A,F=V) = \sum_P \sum_E \ \sum_C P(F=V)P(E)P(P|F=V,E)P(C|E)P(A|P)$$
  
Podemos reescribir la expresión anterior de una forma conveniente para reducir el número de cálculos:  
  
$$
    P(A,F=V) = P(F=V)\sum_P P(A|P) \sum_E P(P|F=V,E)P(E)\sum_C P(C|E)
$$
Factores iniciales:
$$
\begin{align}
P(F=V) &\rightarrow f_F\\
P(A|P) &\rightarrow f_A(A,P)\\
P(P|F=V,E) &\rightarrow f_P(P,E)\\
P(E) &\rightarrow f_E(E)\\
P(C|E) &\rightarrow f_C(C,E)\\ 
\end{align}
$$
Podemos obviar el factor f_F, ya que esta fuera de las sumatorias y es solo una constante que estará incluída en la normalización.  
A continuación los factores: 
  
**Factor $f_A(A,P)$:**

| P | A | Prob |
|---|---|------|
| A | A | 0.75 |
| A | B | 0.25 |
| B | A | 0.05 |
| B | B | 0.95 |

**Factor $f_P(P,E)$:**

| E | P | Prob |
|---|---|------|
| V | A | 0.45 |
| V | B | 0.55 |
| F | A | 0.95 |
| F | B | 0.05 |

**Factor $f_E(E)$:**

| E | Prob |
|---|------|
| V | 0.4  |
| F | 0.6  |

**Factor $f_C(C,E)$:**

| E | C | Prob |
|---|---|------|
| V | A | 0.4 |
| V | B | 0.6 |
| F | A | 0.8 |
| F | B | 0.2 |

Por simplicidad se listará la secuencia de cálculos a realizar para obtener la tabla resultante y se listarán las tablas resultantes de cada paso:

$$
f_{\overline{C}}(E) \times f_P(P,E)\times f_E(E) = f_{P\overline{C}E}(E,P)\\
f_{P\overline{C}\overline{E}}(P)\times f_A(A,P) = f_{P\overline{C}\overline{E}A}(A,P)\\
Resultado = f_{\overline{PCE}A}(A)
$$

A continuación se listan las tablas resultantes de cada cálculos, ya sea marginalizar sobre una variable o hacer el cruce entre dos tablas:  

**Factor $f_{\overline{C}}(E)$:**

| E | Prob |
|---|------|
| V | 1  |
| F | 1  |

**Factor $f_{P\overline{C}}(P,E)$:**

| E | P | Prob |
|---|---|------|
| V | A | 0.45 |
| V | B | 0.55 |
| F | A | 0.95 |
| F | B | 0.05 |

**Factor $f_{P\overline{C}E}(P,E)$:**

| E | P | Prob |
|---|---|------|
| V | A | 0.18 |
| V | B | 0.22 |
| F | A | 0.57 |
| F | B | 0.03 |

**Factor $f_{P\overline{CE}}(P)$:**

| P | Prob |
|---|------|
| A | 0.75  |
| B | 0.25 |

**Factor $f_{P\overline{CE}A}(P,A)$:**

| P | A | Prob |
|---|---|------|
| A | A | 0.5625 |
| A | B | 0.1875 |
| B | A | 0.0125 |
| B | B | 0.2375 |

**Factor $f_{\overline{PCE}A}(P,A)$:**

| A | Prob |
|---|------|
| A | 0.575  |
| B | 0.425 |
  
***Resultado:***  
**Por lo tanto, la probabilidad de que tener un infarto dado que el paciente es fumador es alta con un 57.5% de probabilidad.**

----------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------

### Consulta b 
$$P(P,A=A) = \sum_P \sum_E \ \sum_C P(F)P(E)P(P|F,E)P(C|E)P(A=A|P)$$
  
Podemos reescribir la expresión anterior de una forma conveniente para reducir el número de cálculos:  
  
$$
    P(P,A=A) = P(A=A|P)\sum_EP(E)\sum_CP(C|E)\sum_FP(F)P(P|F,E) 
$$
Factores iniciales:
$$
\begin{align}
P(F) &\rightarrow f_F(F)\\
P(A=A|P) &\rightarrow f_A(P)\\
P(P|F,E) &\rightarrow f_P(P,F,E)\\
P(E) &\rightarrow f_E(E)\\
P(C|E) &\rightarrow f_C(C,E)\\ 
\end{align}
$$
A continuación los factores: 
  
**Factor $f_A(P)$:**

| P | A | Prob |
|---|---|------|
| A | A | 0.75 |
| B | A | 0.05 |

**Factor $f_P(P,F,E)$:**

| E | F | P | Prob |
|---|---|---|------|
| V | V | A | 0.45 |
| V | V | B | 0.55 |
| V | F | A | 0.05 |
| V | F | B | 0.95 |
| F | V | A | 0.95 |
| F | V | B | 0.05 |
| F | F | A | 0.55 |
| F | F | B | 0.45 |

**Factor $f_E(E)$:**

| E | Prob |
|---|------|
| V | 0.4  |
| F | 0.6  |

**Factor $f_C(C,E)$:**

| E | C | Prob |
|---|---|------|
| V | A | 0.4 |
| V | B | 0.6 |
| F | A | 0.8 |
| F | B | 0.2 |

Por simplicidad se listará la secuencia de cálculos a realizar para obtener la tabla resultante y se listarán las tablas resultantes de cada paso:

$$
f_F(F) \times f_P(P,F,E) = f_{FP}(P,F,E)\\
f_{\overline{F}P}(P,E)\times f_C(C,E) = f_{\overline{F}PC}(P,E,C)\\
f_{\overline{F}P\overline{C}}(P,E)\times f_E(E) = f_{\overline{F}P\overline{C}E}(P,E)\\
f_{\overline{F}P\overline{CE}}(P)\times f_A(P) = f_{\overline{F}P\overline{CE}}(P)\\
Resultado = f_{\overline{F}P\overline{CE}A}(P)
$$

A continuación se listan las tablas resultantes de cada cálculos, ya sea marginalizar sobre una variable o hacer el cruce entre dos tablas:  

**Factor $f_{FP}(P,F,E)$:**

| E | F | P | Prob |
|---|---|---|------|
| V | V | A | 0.0675 |
| V | V | B | 0.0825 |
| V | F | A | 0.0425 |
| V | F | B | 0.8075 |
| F | V | A | 0.1425 |
| F | V | B | 0.0075 |
| F | F | A | 0.4675 |
| F | F | B | 0.3825 |

**Factor $f_{\overline{F}P}(P,E)$:**

| E | P | Prob |
|---|---|------|
| V | A | 0.11 |
| V | B | 0.89 |
| F | A | 0.61 |
| F | B | 0.39 |

**Factor $f_{\overline{F}PC}(P,E,C)$:**

| P | E | C | Prob |
|---|---|---|------|
| V | V | A | 0.044 |
| V | V | B | 0.066 |
| V | F | A | 0.488 |
| V | F | B | 0.122 |
| F | V | A | 0.356 |
| F | V | B | 0.534 |
| F | F | A | 0.312 |
| F | F | B | 0.078 |

**Factor $f_{\overline{F}P\overline{C}}(P,E)$:**

| P | E | Prob |
|---|---|------|
| A | V | 0.11 |
| A | F | 0.61 |
| B | V | 0.89 |
| B | F | 0.39 |

**Factor $f_{\overline{F}P\overline{C}E}(P,E)$:**

| P | E | Prob |
|---|---|------|
| A | V | 0.044 |
| A | F | 0.366 |
| B | V | 0.356 |
| B | F | 0.234 |

**Factor $f_{\overline{F}P\overline{CE}}(P)$:**

| P | Prob |
|---|------|
| A | 0.41  |
| B | 0.59 |

**Factor $f_{\overline{F}P\overline{CE}A}(P)$:**

| P | Prob |
|---|------|
| A | 0.3075  |
| B | 0.0295 |

**Factor $f_{\overline{F}P\overline{CE}A}(P)$ NORMALIZADO:**

| P | Prob |
|---|------|
| A | 0.9125 |
| B | 0.0875 |
  
***Resultado:***  
**Por lo tanto, la probabilidad de que tener la presión alta dado que el paciente ha tenido un infarto es de un 91.25%.**

----------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------

### Consulta C 
$$P(A,F=V,C=A) = \sum_P \sum_E \ \sum_C P(F=V)P(E)P(P|F=V,E)P(C=A|E)P(A|P)$$
  
Podemos reescribir la expresión anterior de una forma conveniente para reducir el número de cálculos:  
  
$$
    P(A,F=V,C=A) = P(F=V)\sum_E P(E)P(C=A|E) \sum_P P(P|F=V,E)P(A|P)
$$
Factores iniciales:
$$
\begin{align}
P(F=V) &\rightarrow f_F\\
P(A|P) &\rightarrow f_A(A,P)\\
P(P|F=V,E) &\rightarrow f_P(P,E)\\
P(E) &\rightarrow f_E(E)\\
P(C=A|E) &\rightarrow f_C(E)\\ 
\end{align}
$$
Podemos obviar el factor f_F, ya que esta fuera de las sumatorias y es solo una constante que estará incluída en la normalización.  
A continuación los factores: 
  
**Factor $f_A(A,P)$:**

| P | A | Prob |
|---|---|------|
| A | A | 0.75 |
| A | B | 0.25 |
| B | A | 0.05 |
| B | B | 0.95 |

**Factor $f_P(P,E)$:**

| E | P | Prob |
|---|---|------|
| V | A | 0.45 |
| V | B | 0.55 |
| F | A | 0.95 |
| F | B | 0.05 |

**Factor $f_E(E)$:**

| E | Prob |
|---|------|
| V | 0.4  |
| F | 0.6  |

**Factor $f_C(E)$:**

| E | C | Prob |
|---|---|------|
| V | A | 0.4 |
| F | A | 0.8 |

Por simplicidad se listará la secuencia de cálculos a realizar para obtener la tabla resultante y se listarán las tablas resultantes de cada paso:

$$
f_A(A,P) \times f_P(P,E) = f_{AP}(A,E,P)\\
f_{A\overline{P}}(A,E)\times f_C(E) = f_{A\overline{P}C}(A,E)\\
f_{A\overline{P}C}(A,E)\times f_E(E) = f_{A\overline{P}CE}(A,E)\\
Resultado = f_{A\overline{P}C\overline{E}}(A)
$$

A continuación se listan las tablas resultantes de cada cálculos, ya sea marginalizar sobre una variable o hacer el cruce entre dos tablas:  

**Factor $f_{AP}(A,E,P)$:**


| P | A | E | Prob |
|---|---|---|------|
| A | A | V | 0.3375 |
| A | A | F | 0.7125 |
| A | B | V | 0.1125 |
| A | B | F | 0.2375 |
| B | A | V | 0.0275 |
| B | A | F | 0.0025 |
| B | B | V | 0.5225 |
| B | B | F | 0.0475 |

**Factor $f_{A\overline{P}}(A,E)$:**

| A | E | Prob |
|---|---|------|
| A | V | 0.365 |
| A | F | 0.715 |
| B | V | 0.635 |
| B | F | 0.285 |

**Factor $f_{A\overline{P}C}(A,E)$:**

| A | E | Prob |
|---|---|------|
| A | V | 0.146 |
| A | F | 0.572 |
| B | V | 0.254 |
| B | F | 0.228 |

**Factor $f_{A\overline{P}CE}(A,E)$:**

| A | E | Prob |
|---|---|------|
| A | V | 0.0584 |
| A | F | 0.3432 |
| B | V | 0.1016 |
| B | F | 0.1368 |

**Factor $f_{A\overline{P}C\overline{E}}(A)$:**

| A | Prob |
|---|------|
| A | 0.4016  |
| B | 0.2384 |

**Factor $f_{A\overline{P}C\overline{E}}(A)$ NORMALIZADO:**

| A | Prob |
|---|------|
| A | 0.6275 |
| B | 0.3725 |
  
***Resultado:***  
**Por lo tanto, la probabilidad de que tener un infarto dado que el paciente es fumador y tiene el colesterol alto es alta con un 62.75% de probabilidad.**

## 3. Código

In [ ]:
enfermedad = BayesianModel([("F", "P"), ("E", "P"), ("E", "C"), ("P", "A")])

F_cpd = TabularCPD("F", 2, [[0.15], [0.85]], state_names={"F": ["V", "F"]})
E_cpd = TabularCPD("E", 2, [[0.4], [0.6]], state_names={"E": ["V", "F"]})
P_cpd = TabularCPD(
    "P",
    2,
    [[0.45, 0.95, 0.05, 0.55], [0.55, 0.05, 0.95, 0.45]],
    evidence=["F", "E"],
    evidence_card=[2, 2],
    state_names={"P": ["A", "B"], "F": ["V", "F"], "E": ["V", "F"]},
)
C_cpd = TabularCPD(
    "C",
    2,
    [[0.4, 0.8], [0.6, 0.2]],
    evidence=["E"],
    evidence_card=[2],
    state_names={"C": ["A", "B"], "E": ["V", "F"]},
)
A_cpd = TabularCPD(
    "A",
    2,
    [[0.75, 0.05], [0.25, 0.95]],
    evidence=["P"],
    evidence_card=[2],
    state_names={"A": ["A", "B"], "P": ["A", "B"]},
)
enfermedad.add_cpds(F_cpd, E_cpd, P_cpd, C_cpd, A_cpd)


In [ ]:
infer2 = VariableElimination(enfermedad)
print(infer2.query(["A"], evidence={"F": "V"}))


Eliminating: C: 100%|███████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 333.74it/s]

+------+----------+
| A    |   phi(A) |
+======+==========+
| A(A) |   0.5750 |
+------+----------+
| A(B) |   0.4250 |
+------+----------+


In [ ]:
print(infer2.query(["P"], evidence={"A": "A"}))


Eliminating: E: 100%|███████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 428.95it/s]

+------+----------+
| P    |   phi(P) |
+======+==========+
| P(A) |   0.9125 |
+------+----------+
| P(B) |   0.0875 |
+------+----------+


In [ ]:
print(infer2.query(["A"], evidence={"F": "V", "C": "A"}))


Eliminating: E: 100%|███████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 286.64it/s]

+------+----------+
| A    |   phi(A) |
+======+==========+
| A(A) |   0.6275 |
+------+----------+
| A(B) |   0.3725 |
+------+----------+


Como se observa se obtuvieron los mismos resultados.